In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Numeric, DateTime, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref
from datetime import datetime
from sqlalchemy.orm import sessionmaker

In [2]:
Base = declarative_base()

class Cookie(Base):
    __tablename__ = "cookies"

    cookie_id = Column(Integer(), primary_key=True)
    cookie_name = Column(String(50), index=True)
    cookie_receipe_url = Column(String(255))
    cookie_sku = Column(String(55))
    quantity = Column(Integer())
    unit_cost = Column(Numeric(12, 2))

    def __repr__(self):
        return f"<Cookie: {self.cookie_name}>"


class User(Base):
    __tablename__ = "users"

    user_id = Column(Integer(), primary_key=True)
    username = Column(String(15), nullable=False, unique=True)
    email_address = Column(String(255), nullable=False)
    phone = Column(String(20), nullable=False)
    password = Column(String(50), nullable=False)
    created_on = Column(DateTime(), default=datetime.now)
    updated_on = Column(DateTime(), default=datetime.now, onupdate=datetime.now)

    def __repr__(self):
        return f"<User: {self.username}>"


class Order(Base):
    __tablename__ = "orders"

    order_id = Column(Integer(), primary_key=True)
    user_id = Column(Integer, ForeignKey("users.user_id"))
    shipped = Column(Boolean, default=False)

    user = relationship("User", backref=backref("orders", order_by=order_id))

    def __repr__(self):
        return f"<Order: {self.order_id}>" 

class LineItem(Base):
    __tablename__ = "line_items"

    line_item_id = Column(Integer(), primary_key=True)
    order_id = Column(Integer(), ForeignKey("orders.order_id"))
    cookie_id = Column(Integer(), ForeignKey("cookies.cookie_id"))
    quantity = Column(Integer())
    extended_cost = Column(Numeric(12, 2))

    cookie = relationship("Cookie", uselist=False, backref=backref("line_items"), order_by=line_item_id)
    order = relationship("Order", backref=backref("line_items"), order_by=line_item_id)

    def __repr__(self):
        return f"<LineItem: {self.line_item_id}>"


engine = create_engine("sqlite:///:memory:")
Base.metadata.create_all(engine)

In [3]:
# create a session - it acts like a transaction

Session = sessionmaker(bind=engine)
session = Session()


In [4]:
# inserting data

chocolate_chip = Cookie(cookie_name="chocolate chip",
                        cookie_receipe_url="htts://somewebsite.com",
                        cookie_sku="CC01",
                        quantity=12,
                        unit_cost=0.5)

session.add(chocolate_chip)
session.commit()
# print(chocolate_chip.cookie_id) # primary key is updated on inserted objects

d_chocolate_chip = Cookie(
                            cookie_name="dark chocolate chip",
                            cookie_receipe_url="htts://somewebsite.com/2",
                            cookie_sku="CC02",
                            quantity=16,
                            unit_cost=0.8
)

molasis = Cookie(
                            cookie_name="molasis",
                            cookie_receipe_url="htts://somewebsite.com/3",
                            cookie_sku="CC03",
                            quantity=5,
                            unit_cost=0.4
)

session.add(d_chocolate_chip)
session.add(molasis)
session.flush()  # send data to db BUT DO NOT COMMIT TRANSACTION!! i.e. must be commited later
# print(molasis.cookie_id)  # even though the transaction has not been commited, the objects are updated with primary keys


peanut_butter = Cookie(
                            cookie_name="peanut butter",
                            cookie_receipe_url="htts://somewebsite.com/4",
                            cookie_sku="CC04",
                            quantity=3,
                            unit_cost=0.9
)

oatmeal_raisin = Cookie(
                            cookie_name="oatmeal raisin",
                            cookie_receipe_url="htts://somewebsite.com/5",
                            cookie_sku="CC05",
                            quantity=15,
                            unit_cost=0.35
)
session.bulk_save_objects([peanut_butter, oatmeal_raisin])
session.commit()
# print(oatmeal_raisin.cookie_id)  # ids not returned but is faster




In [5]:
# Querying data

all_cookies = session.query(Cookie).all()  # all records

# print(all_cookies)  # list of cookie objects
# [cookie for cookie in session.query(Cookie)]  # query can be used as iterable


cookie_names = session.query(Cookie.cookie_name)  # specific column

# print([name for name in cookie_names]) # returns iterable of tuples
# print([name.cookie_name for name in cookie_names])  # accesses the string we are looking for


res = session.query(Cookie.cookie_name, Cookie.quantity)  # multiple columns

# print([row for row in res])




/home/rory/repos/sql_book/venv/lib/python3.7/site-packages/sqlalchemy/sql/sqltypes.py:666: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  "storage." % (dialect.name, dialect.driver)


In [6]:
# Ordering
from sqlalchemy.sql import desc

by_quantity = session.query(Cookie.cookie_name, Cookie.quantity).order_by(Cookie.quantity)  # ascending

# print([row for row in by_quantity])

by_quantity_desc = session.query(Cookie.cookie_name, Cookie.quantity).order_by(desc(Cookie.quantity))  # descending

# print([row for row in by_quantity_desc])


In [7]:
# Limiting

res = session.query(Cookie.cookie_name).limit(3)
# print([r for r in res])

In [8]:
# Applying SQL functions

from sqlalchemy import func


total_cookies = session.query(func.sum(Cookie.quantity))
# print(total_cookies.scalar())


res = session.query(func.count(Cookie.cookie_id))
# print(res.scalar())


In [9]:
# Labelling columns

res = session.query(func.count(Cookie.cookie_name).label("total cookie types"))
print(res.scalar())

5


In [10]:
# Filtering

res = session.query(Cookie).filter(Cookie.cookie_name == "chocolate chip")
# print(res.first())

res = session.query(Cookie.cookie_receipe_url).filter(Cookie.cookie_id == 1)
# print(res.first())

res = session.query(Cookie.cookie_name).filter(Cookie.cookie_name.like("%Chocolate%"))

# print([r for r in res])


## filter by

res = session.query(Cookie).filter_by(cookie_name="chocolate chip")
# print([r for r in res])


In [11]:
# Logical operators

from sqlalchemy.sql import not_, or_, and_

res = session.query(Cookie.cookie_name, Cookie.cookie_id).filter(not_(Cookie.cookie_name == "chocolate chip"))
# print([r for r in res])

res = session.query(Cookie.cookie_name).filter(or_(Cookie.unit_cost < 0.5, Cookie.quantity > 10))
# print(*res)



In [12]:
# Arithmetic operators

res = session.query(Cookie.cookie_name, 'SKU-' + Cookie.cookie_sku).limit(3)
# print(*res)

res = session.query(Cookie.cookie_name, (Cookie.quantity > 0).label("in_stock"))
# print(*res)

In [13]:
# Type casting

from sqlalchemy.sql import cast

res = session.query(Cookie.cookie_name, cast((Cookie.quantity * Cookie.unit_cost), Numeric(12, 2)).label('total_value'))
# print(*res)

In [14]:
# Updating data

# method 1: query to create an instance, update an attribute then commit the session

chocolate_chip = session.query(Cookie)\
                        .filter(Cookie.cookie_name == "chocolate chip")\
                        .first()
chocolate_chip.quantity += 10
session.commit()


# method 2: update directly from the query

session.query(Cookie)\
        .filter(Cookie.cookie_name == "chocolate chip")\
        .update({Cookie.quantity: Cookie.quantity - 20})

1

In [15]:
# Deleting data

# method 1: query to get a session-bound object, then update its attributes and commit the session

plentiful_cookies = session.query(Cookie).filter(Cookie.quantity > 15).all()
[session.delete(cookie) for cookie in plentiful_cookies]
session.commit()

# method 2: directly delete from the query

session.query(Cookie).filter(Cookie.cookie_name == "molasis").delete()

1

In [16]:
# loading data for joins

# users

cookiemon = User(username="cookiemon",
                 email_address="mon@cookie.com",
                 phone="111-111-1111",
                 password="password")
cakeater = User(username="cakeater",
                 email_address="cakeater@cakeater.com",
                 phone="222-222-2222",
                 password="password")
pieperson = User(username="pieperson",
                 email_address="pie@person.com",
                 phone="333-333-3333",
                 password="password")

session.add(cookiemon)
session.add(cakeater)
session.add(pieperson)
session.commit()

# orders
order_1 = Order()
order_1.user = cookiemon
session.add(order_1)

chocolate_chip = session.query(Cookie).filter(Cookie.cookie_name == "chocolate chip").first()
peanut_butter = session.query(Cookie).filter(Cookie.cookie_name == "peanut butter").first()

# line items
line_item_1 = LineItem(
    cookie=chocolate_chip,
    quantity=2, 
    extended_cost=1.00,
    order=order_1)
order_1.line_items.append(line_item_1)  # can do this instead of

line_item_2 = LineItem(
    cookie=peanut_butter,
    quantity=12,
    order=order_1,
    extended_cost=3.00)

session.commit()

# o1 = session.query(Order).filter(Order.user == cookiemon).first()
# print(o1.line_items)

order_2 = Order()
order_2.user = cakeater

chocolate_chip = session.query(Cookie).filter(Cookie.cookie_name == "chocolate chip").first()
line_item_1 = LineItem(cookie=chocolate_chip, quantity=3, extended_cost=8)

oat = session.query(Cookie).filter(Cookie.cookie_name.like("%oat%")).first()
line_item_2 = LineItem(cookie=oat, quantity=2, extended_cost=5)

order_2.line_items.append(line_item_1)
order_2.line_items.append(line_item_2)

session.add(order_2)
session.commit()

# print(session.query(LineItem).all())  # line items were added too 
                

In [26]:
# Joins

res = session.query(
                        Order.order_id, User.username, User.phone,
                        Cookie.cookie_name, LineItem.quantity, LineItem.extended_cost
                    )\
             .join(User)\
             .join(LineItem)\
             .join(Cookie)\
             .filter(User.username == "cookiemon")\
             .all()
# print(*res)


res = session.query(User.username, func.count(Order.order_id))\
             .outerjoin(Order)\
             .group_by(User.username)
print(*res)

('cakeater', 1) ('cookiemon', 1) ('pieperson', 0)


In [37]:
# Chaining

def get_orders_by_customer(cust_name):
    res = session.query(Order.order_id, User.username, User.phone, Cookie.cookie_name, LineItem.quantity)\
                 .join(User).join(LineItem).join(Cookie)\
                 .filter(User.cust_name == cust_name)\
                 .all()
    return res


# [or]

def get_orders_by_customer(cust_name, shipped=None, details=False):
    q = session.query(Order.order_id, User.user_id, User.phone).join(User)
    if details:
        q = q.add_columns(Cookie.cookie_name, LineItem.quantity)
        q = q.join(LineItem).join(Cookie)
    if shipped is not None:
        q = q.filter(Order.shipped == shipped)
    res = q.filter(User.username == cust_name).all()
    return res


print(get_orders_by_customer("cakeater", details=True, shipped=False))

[(2, 2, '222-222-2222', 'chocolate chip', 3), (2, 2, '222-222-2222', 'oatmeal raisin', 2)]


In [42]:
# Raw Queries

from sqlalchemy import text

q = session.query(Cookie).filter(text("cookies.cookie_name == 'chocolate chip'"))
print(*q)


<Cookie: chocolate chip>
